# Create .PNG images of all timesteps in ECCC PM2.5 dataset

## Import libraries and data

In [1]:
# for numerical work
import numpy as np
import itertools

# for accessing file system
import os

# for loading netcdf files, for metadata
import xarray as xr
# for connecting OpenVisus framework to xarray
# from https://github.com/sci-visus/openvisuspy, 
from openvisuspy.xarray_backend import OpenVisusBackendEntrypoint

# Used for processing netCDF time data
import time
import datetime
import requests
# Used for indexing via metadata
import pandas as pd

# for plotting
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs

# for parallelization
import multiprocessing

#Stores the OpenVisus cache in the local direcrtory 
import os
os.environ["VISUS_CACHE"]="./visus_cache_can_be_erased"
os.environ['CURL_CA_BUNDLE'] = ''

### Import ECCC 2021 and 2022 Data

In [2]:
df_eccc = pd.read_csv('PM25_2021_2022.csv')

### Create set of all hours to query for

In [3]:
eccc_dates = df_eccc['Date//Date'].values
all_dates = []
for d in eccc_dates:
    for i in range(24):
        all_dates.append(pd.Timestamp(f'{d} {i}:00:00'))

In [4]:
for i in enumerate(all_dates[0:4]):
    print(i)

(0, Timestamp('2021-01-01 00:00:00'))
(1, Timestamp('2021-01-01 01:00:00'))
(2, Timestamp('2021-01-01 02:00:00'))
(3, Timestamp('2021-01-01 03:00:00'))


### Plot ECCC Smoke Emissions over time

In [5]:
df_eccc

,Unnamed: 0,Latitude//Latitude,Longitude//Longitude,Date//Date,0,1,2,3,4,5,...,14,15,16,17,18,19,20,21,22,23
0,0,45.64103,-73.49968,2021-01-01,15.0,10.0,4.0,1.0,8.0,11.0,...,5.0,5.0,6.0,7.0,10.0,8.0,7.0,9.0,9.0,8.0
1,1,45.64103,-73.49968,2021-01-02,7.0,8.0,8.0,8.0,7.0,7.0,...,4.0,7.0,12.0,10.0,12.0,16.0,18.0,15.0,11.0,9.0
2,2,45.64103,-73.49968,2021-01-03,11.0,10.0,12.0,12.0,16.0,14.0,...,10.0,11.0,17.0,28.0,32.0,31.0,30.0,23.0,22.0,20.0
3,3,45.64103,-73.49968,2021-01-04,16.0,15.0,15.0,15.0,15.0,14.0,...,16.0,16.0,20.0,20.0,23.0,30.0,30.0,27.0,18.0,15.0
4,4,45.64103,-73.49968,2021-01-05,12.0,16.0,18.0,20.0,20.0,17.0,...,3.0,3.0,5.0,5.0,4.0,5.0,8.0,8.0,7.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154081,93070,60.00455,-111.89324,2022-12-27,1.0,2.0,1.0,2.0,1.0,1.0,...,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0
154082,93071,60.00455,-111.89324,2022-12-28,2.0,2.0,1.0,3.0,1.0,1.0,...,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0
154083,93072,60.00455,-111.89324,2022-12-29,4.0,3.0,3.0,2.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0
154084,93073,60.00455,-111.89324,2022-12-30,1.0,2.0,3.0,1.0,1.0,1.0,...,1.0,1.0,2.0,2.0,2.0,5.0,5.0,5.0,4.0,4.0


In [24]:
save_dir = "/usr/sci/scratch_nvme/arleth/dump/eccc_frames/"
my_norm = "log"
my_cmap = "autumn"
my_vmin = 1e-13
my_vmax = 10e4
fig_w, fig_h = 15, 6

def create_frame_catch_issues(frame_date_tuple):
    # frame number to save PNG as and date to visualize
    frame_num = frame_date_tuple[0]
    date = frame_date_tuple[1]
    hour = date.hour

    # set figure size
    my_fig, my_plt = plt.subplots(figsize=(fig_w, fig_h), subplot_kw=dict(projection=ccrs.PlateCarree()))
    
    # select datapoints for given date
    date_cond = df_eccc['Date//Date'] == pd.Timestamp(day=date.day, month=date.month, year=date.year).strftime('%Y-%m-%d')

    # get the values for the given hour and get latitudes and longitudes for plotting
    curr_vals = df_eccc[date_cond][f'{hour}'].values
    curr_lats = df_eccc[date_cond]['Latitude//Latitude'].values
    curr_lons = df_eccc[date_cond]['Longitude//Longitude'].values

    # use basemap to plot values: https://basemaptutorial.readthedocs.io/en/latest/plotting_data.html#scatter
    # use `cyl` project: https://matplotlib.org/basemap/stable/users/cyl.html
    # set parameters: https://basemaptutorial.readthedocs.io/en/latest/basemap.html
    m = Basemap(projection='cyl', llcrnrlat=np.min(curr_lats),urcrnrlat=np.max(curr_lats),
            llcrnrlon=np.min(curr_lons),urcrnrlon=np.max(curr_lons), resolution='l',
            fix_aspect=False, area_thresh=1e6)
    
    my_fig.suptitle(f'Ground level concentration of PM2.5 microns and smaller {date}\n')
    # add caption showing this is from ECCC dataset
    my_fig.text(0.5, 0, 'ECCC Data', ha='center', va='center')
    
    # Draw map features
    m.drawcoastlines()
    m.drawparallels(np.arange(45.,66.,5.),labels=[1,1,1,1]) # draw parallels
    m.drawmeridians(np.arange(-120.,-59.,20.),labels=[1,1,1,1]) # draw parallels

    # Convert lat/lon to map coordinates for Basemap scatter plot
    x, y = m(curr_lons, curr_lats)

    # Plot the lats and lons
    sc = m.scatter(x, y, c=curr_vals, marker='o', s=3.5, cmap=my_cmap, 
                 norm=my_norm, vmin=my_vmin, vmax=my_vmax)

    plt.colorbar(sc,location='right', label='ug/m^3')
    # save visualization as a .PNG to our folder
    plt.savefig(save_dir + "frames%05d.png" % frame_num, dpi=280)
    plt.close(my_fig);  # close the figure after saving
    plt.show()

In [25]:
 # create frames, capturing issues 
with multiprocessing.Pool() as pool:
    # Start a timer to measure how long the conversion takes
    start_time = time.time()
    print('starting')
    issues = pool.map(create_frame_catch_issues, enumerate(all_dates[0:4]))
    print('done!')
    # End the timer and print the elapsed time
    end_time = time.time()
    print(f'Total elapsed time: {end_time - start_time}')

starting
done!
Total elapsed time: 3.1821231842041016
